In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import pandas as pd

In [6]:
# # Não faz parte
# pd.read_csv('https://github.com/wcota/covid19br/raw/master/cases-brazil-cities-time_changesOnly.csv.gz').reset_index().to_csv('/tmp/cases-brazil-cities-time.csv', index=False)

### Leitura de dados

In [3]:
read_path = 'cases-brazil-cities-time.csv'

In [16]:
df = spark.read.csv(read_path, header=True, sep=',', multiLine=True)

### Visualizando dados, quantidade de registros e de colunas

In [20]:
df.show()

+-----+--------+----------+-------+-----+--------------------+-------+---------+------+--------+----------+---------------------------+-------------------------------+--------------------+-------+--------------+
|index|epi_week|      date|country|state|                city| ibgeID|newDeaths|deaths|newCases|totalCases|deaths_per_100k_inhabitants|totalCases_per_100k_inhabitants|deaths_by_totalCases|_source|last_info_date|
+-----+--------+----------+-------+-----+--------------------+-------+---------+------+--------+----------+---------------------------+-------------------------------+--------------------+-------+--------------+
|    0|       9|2020-02-25| Brazil|   SP|        São Paulo/SP|3550308|        0|     0|       1|         1|                        0.0|                        0.00811|                 0.0|    SES|    2021-07-03|
|    1|       9|2020-02-25| Brazil|TOTAL|               TOTAL|      0|        0|     0|       1|         1|                        0.0|                 

In [22]:
tot = df.count()
print(tot)

1140545


In [23]:
# NÃO FAZER ISSO EM TODO O DATAFRAME
df.toPandas()

,index,epi_week,date,country,state,city,ibgeID,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,last_info_date
0,0,9,2020-02-25,Brazil,SP,São Paulo/SP,3550308,0,0,1,1,0.0,0.00811,0.0,SES,2021-07-03
1,1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,0,0,1,1,0.0,0.00047,0.0,None,None
2,2,9,2020-02-28,Brazil,SP,São Paulo/SP,3550308,0,0,1,2,0.0,0.01623,0.0,SES,2021-07-03
3,3,9,2020-02-28,Brazil,TOTAL,TOTAL,0,0,0,1,2,0.0,0.00094,0.0,None,None
4,4,10,2020-03-04,Brazil,SP,São Paulo/SP,3550308,0,0,1,3,0.0,0.02434,0.0,SES,2021-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140540,1140540,127,2021-07-05,Brazil,SC,CASO SEM LOCALIZAÇÃO DEFINIDA/SC,42,50,50,1313,1313,0.69786,18.32573,0.03808,GI,2021-07-05
1140541,1140541,127,2021-07-05,Brazil,SE,CASO SEM LOCALIZAÇÃO DEFINIDA/SE,28,9,9,348,348,0.39153,15.13902,0.02586,GI,2021-07-05
1140542,1140542,127,2021-07-05,Brazil,SP,CASO SEM LOCALIZAÇÃO DEFINIDA/SP,35,66,66,4231,5165,0.14373,11.24806,0.01278,GI,2021-07-05
1140543,1140543,127,2021-07-05,Brazil,TO,CASO SEM LOCALIZAÇÃO DEFINIDA/TO,17,6,6,195,195,0.38147,12.39775,0.03077,GI,2021-07-05


In [44]:
# USEM ASSIM
df.limit(20).toPandas()

,index,epi_week,date,country,state,city,ibgeID,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,last_info_date
0,0,9,2020-02-25,Brazil,SP,São Paulo/SP,3550308,0,0,1,1,0.0,0.00811,0.0,SES,2021-07-03
1,1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,0,0,1,1,0.0,0.00047,0.0,None,None
2,2,9,2020-02-28,Brazil,SP,São Paulo/SP,3550308,0,0,1,2,0.0,0.01623,0.0,SES,2021-07-03
3,3,9,2020-02-28,Brazil,TOTAL,TOTAL,0,0,0,1,2,0.0,0.00094,0.0,None,None
4,4,10,2020-03-04,Brazil,SP,São Paulo/SP,3550308,0,0,1,3,0.0,0.02434,0.0,SES,2021-07-03


In [45]:
# tipos das colunas
df.printSchema()

root
 |-- index: string (nullable = true)
 |-- epi_week: string (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- ibgeID: string (nullable = true)
 |-- newDeaths: string (nullable = true)
 |-- deaths: string (nullable = true)
 |-- newCases: string (nullable = true)
 |-- totalCases: string (nullable = true)
 |-- deaths_per_100k_inhabitants: string (nullable = true)
 |-- totalCases_per_100k_inhabitants: string (nullable = true)
 |-- deaths_by_totalCases: string (nullable = true)
 |-- _source: string (nullable = true)
 |-- last_info_date: string (nullable = true)



In [47]:
# Quantidade de colunas
len(df.columns)

16

### Seleção de colunas

In [48]:
print(df.columns)

['index', 'epi_week', 'date', 'country', 'state', 'city', 'ibgeID', 'newDeaths', 'deaths', 'newCases', 'totalCases', 'deaths_per_100k_inhabitants', 'totalCases_per_100k_inhabitants', 'deaths_by_totalCases', '_source', 'last_info_date']


In [54]:
lista_colunas = ['index', 'date', 'ibgeID', 'newCases']
df.select(lista_colunas).show(5)

# # Outra forma
# df.select('index', 'date', 'ibgeID', 'newCases')

+-----+----------+-------+--------+
|index|      date| ibgeID|newCases|
+-----+----------+-------+--------+
|    0|2020-02-25|3550308|       1|
|    1|2020-02-25|      0|       1|
|    2|2020-02-28|3550308|       1|
|    3|2020-02-28|      0|       1|
|    4|2020-03-04|3550308|       1|
+-----+----------+-------+--------+
only showing top 5 rows



### Criando colunas - parte 1

In [79]:
df_linkado = df.withColumn('ind_linkado', F.when(F.col('newCases') > 5, F.lit(1)).otherwise(F.lit(0)))

In [83]:
# df_linkado = df.withColumn('ind_linkado', F.when(F.col('score') > 0.5, F.lit(1)).otherwise(F.lit(0)).cast(ByteType()))

In [85]:
df_linkado.select('score', 'ind_linkado').show()

+-------------------+-----------+
|              score|ind_linkado|
+-------------------+-----------+
| 0.6949237848667377|          1|
|0.09451786679678054|          0|
| 0.9252092317484667|          1|
| 0.8302538645990428|          1|
| 0.5763907027649712|          1|
|0.05812679609412996|          0|
| 0.7159847065041175|          1|
| 0.3898762178949218|          0|
|0.43118243527290756|          0|
|  0.440226218321238|          0|
| 0.2085144499270134|          0|
| 0.6326891341644962|          1|
| 0.2872127669203912|          0|
| 0.3653878880646141|          0|
|0.21549028984793983|          0|
| 0.8372403504356234|          1|
| 0.7741113327657877|          1|
| 0.4143334921355972|          0|
|0.04440248824406878|          0|
|  0.420776049031671|          0|
+-------------------+-----------+
only showing top 20 rows



### Summary

In [86]:
df_linkado.select('date', 'index', 'newCases', 'epi_week', 'ind_linkado').summary().toPandas()

,summary,date,index,newCases,epi_week,ind_linkado
0,count,1140545,1140545,1140545,1140545,1140545
1,mean,None,570272.0,32.963353484518365,75.62785247403653,0.5001836841159271
2,stddev,None,329247.12572397647,959.3165412847385,39.7896556768549,0.5000001854537331
3,min,2020-02-25,0,-1,10,0
4,25%,None,285076.0,1.0,36.0,0
5,50%,None,570197.0,4.0,101.0,1
6,75%,None,855299.0,11.0,114.0,1
7,max,2021-07-05,999999,998,9,1


____

### Filtrando dados

### Extraindo dados de colunas
* Dia, mes, ano
* Extraindo o estado do municipio

### Operações entre colunas 
* +, -, *, /

### Comparando colunas
* Colunas de linkage
* Registros com score igual a 1